In [ ]:
#| eval: false
! pip install pyzotero
! pip install git+https://github.com/JoFrhwld/zotero2qmd.git

`zoetero2qmd` is a quick and dirty package I wrote to convert the output from `pyzotero` to compatible data structures for quarto headers.

In [2]:
from pyzotero import zotero
from zotero2qmd.zotero2qmd import item2main, main2qmd, filter_pubs
from collections import Counter
import re
import yaml
from pathlib import Path

If you are reusing this code, you'll need to get your own API key: [pyzotero getting started](https://pyzotero.readthedocs.io/en/latest/#getting-started-short-version)

In [3]:
keypath = Path(".zotero")
with keypath.open() as keyfile:
    key_text = keyfile.readline().strip()
zot = zotero.Zotero(library_id='7642731', library_type='user', api_key=key_text)    

The `zotero.Zotero.publications()` method pulls down all citations you've added to "My Publications".

In [4]:
pubs = zot.publications()

Everything that isn't a talk is going into one big pile for now. I'll figure out what to do with talks later. Also, I don't know why I started using "main" to refer to the publication info.

In [5]:
filtered_pubs = filter_pubs(pubs)
all_mains = [item2main(x) for x in filtered_pubs]

Each dictionary can be yaml dumped into valid quarto headings.

In [6]:
all_mains[0]

{'params': {'key': 'BNWT43F9', 'notes': 'DOI: 10.5281/ZENODO.10212099'},
 'author': [{'name': {'given': 'Josef', 'family': 'Fruehwald'}},
  {'name': {'given': 'Santiago', 'family': 'Barreda'}}],
 'title': 'fasttrackpy',
 'date': '2023-11-28',
 'date-format': 'YYYY',
 'description': 'v0.3.0',
 'abstract': 'This is a python implementation of the FastTrack method.',
 'citation': {'type': 'software',
  'issued': '2023-11-28',
  'url': 'https://fasttrackiverse.github.io/fasttrackpy/',
  'version': 'v0.3.0'}}

It took a bit of work to generate mostly legible but unique file names. I decided to append the Zotero `key` value to each.

In [7]:
def make_file_names(all_mains):
    first_aut = [x["author"][0]["name"]["family"] for x in all_mains]
    years = [x["date"].split("-")[0] for x in all_mains]
    keys = [x["params"]["key"] for x in all_mains]

    all_stem = [f"{aut}_{year}_{key}" for aut, year, key in zip(first_aut, years, keys)]

    all_stem = [re.sub(r"_$", "", x) for x in all_stem]
    return all_stem

In [8]:
all_stems = make_file_names(all_mains)

I started dumping everything into "papers"... so now I'm stuck with it.

In [9]:
out_path = Path("research","papers")

I only want to write new files, since I may manually edit the results here and there.

In [10]:
for stem, item in zip(all_stems, all_mains):
    out_file = out_path.joinpath(stem).with_suffix(".qmd")
    if not out_file.exists():
        with out_file.open(mode = "w"):
            qmd_string = "---\n"+yaml.dump(item)+"\n---"
            out_file.write_text(qmd_string)